# **Monstrinho 3.8**

* Caio M. C. Ruas - RM: 24010

## Introdução

O objetivo deste monstrinho é utilizar um algorítmo genético para encontra as coordenadas (x, y) dos mínimos globais da função de Himmelblau abaixo.

$$ f(x, y) = (x^2 + y - 11)^2 + (x + y^2 - 7)^2 $$

Como estratégia para resolver esse problema, foi programado um algoritmo genético que cria indivíduos com duas características x e y, referentes as próprias coordenadas. Para serem criados, foi feita uma escolha aleatória de x e y entre os valores -10 e 10. Essa margem foi escolhida devido as características da função de Himmelblau, se analisarmos, é possível perceber que os termos entre parênteses são quadrados, o que significa que a função só pode assumir valores positivos, portanto faz sentido que os mínimos globais sejam quando esses termos forem iguais a zero. Assim, faz sentido escolher números da escala dos "termos independentes" que aparecem (-11 e -7), sendo assim, -10 e 10.

A cada geração, os indivíduos são avaliados através da própria função de Himmelblau, onde o menor valor encontrado é considerado o melhor indivíduo. 

Para a seleção dos indivíduos, foi utilizado o método de torneio, onde três indivíduos são escolhidos aleatoriamente e o que possui o menor valor da função é selecionado para a próxima geração.

Após a seleção, os indivíduos são cruzados com a probabilidade de 0.1 pelo método de cruzamento uniforme, onde cada característica é escolhida aleatoriamente entre os dois indivíduos selecionados. Após o cruzamento, os filhos são mutados com uma probabilidade de 0.1, onde a mutação consiste em adicionar um valor aleatório entre -0.5 e 0.5 a cada característica do filho.

O código segue abaixo, primeiramente definiremos as funções necessárias para o funcionamento do algoritmo, e logo após, a função principal que executa o algoritmo genético.

In [1]:
# --- Bibliotecas ---

import random
import numpy as np

# --- Funções ---

# Definindo a semente para reprodutibilidade
random.seed(42)

def gene_him():
    """Gera um número aleatório entre o intervalo colocado"""
    return random.uniform(-10, 10)

def criar_candidato_him():
    """Cria um candidato com dois genes"""
    return [gene_him(), gene_him()]

def criar_populacao_him(tamanho_populacao):
    """Cria uma população de candidatos"""
    return [criar_candidato_him() for _ in range(tamanho_populacao)]

def funcao_him(candidato):
    x = candidato[0]
    y = candidato[1]
    return (x**2 + y - 11)**2 + (x + y**2 - 7)**2

def calcular_fitness_him(populacao):
    """Calcula o fitness de uma populacao"""
    fitness = []
    for individuo in populacao:
        fitness.append(funcao_him(individuo))
    return fitness

def selecao_torneio_min(populacao, fitness, tamanho_torneio):
    """Faz a seleção de uma população usando torneio.

    Nota: da forma que está implementada, só funciona em problemas de
    minimização.

    Args:
      populacao: lista contendo os individuos do problema
      fitness: lista contendo os valores computados da funcao objetivo
      tamanho_torneio: quantidade de invíduos que batalham entre si

    """
    selecionados = []

    for _ in range(len(populacao)):
        sorteados = random.sample(populacao, tamanho_torneio)

        fitness_sorteados = []
        for individuo in sorteados:
            indice_individuo = populacao.index(individuo)
            fitness_sorteados.append(fitness[indice_individuo])

        min_fitness = min(fitness_sorteados)
        indice_min_fitness = fitness_sorteados.index(min_fitness)
        individuo_selecionado = sorteados[indice_min_fitness]

        selecionados.append(individuo_selecionado)

    return selecionados

def cruzamento_uniforme(pai, mae, chance_de_cruzamento):
    """Realiza cruzamento uniforme

    Args:
      pai: lista representando um individuo
      mae: lista representando um individuo
      chance_de_cruzamento: float entre 0 e 1 representando a chance de cruzamento

    """
    if random.random() < chance_de_cruzamento:
        filho1 = []
        filho2 = []

        for gene_pai, gene_mae in zip(pai, mae):
            if random.choice([True, False]):
                filho1.append(gene_pai)
                filho2.append(gene_mae)
            else:
                filho1.append(gene_mae)
                filho2.append(gene_pai)

        return filho1, filho2
    else:
        return pai, mae

def mutacao_simples_him(populacao, chance_de_mutacao, intervalo_numerico):
    """Realiza mutação simples no problema de minimização

    Args:
      populacao: lista contendo os indivíduos do problema
      chance_de_mutacao: float entre 0 e 1 representando a chance de mutação
      intervalo_numerico: inteiro representando o intervalo numérico

    """
    for individuo in populacao:
        if random.random() < chance_de_mutacao:
            gene = random.randint(0, len(individuo) - 1)
            valor_gene = individuo[gene]
            individuo[gene] = valor_gene + random.uniform(-intervalo_numerico, intervalo_numerico)

# --- Função auxiliar ---

def executar_algoritmo_genetico(tamanho_populacao, tamanho_torneio, chance_de_cruzamento, chance_de_mutacao, intervalo_numerico, geracoes):
    """Executa o algoritmo genético para otimização de função

    Args:
      tamanho_populacao: inteiro representando o tamanho da população
      tamanho_torneio: inteiro representando o tamanho do torneio
      chance_de_cruzamento: float entre 0 e 1 representando a chance de cruzamento
      chance_de_mutacao: float entre 0 e 1 representando a chance de mutação
      intervalo_numerico: inteiro representando o intervalo numérico
      geracoes: inteiro representando o número de gerações

    """
    # Inicializa a população e calcula o fitness
    populacao = criar_populacao_him(tamanho_populacao)
    fitness = calcular_fitness_him(populacao)

    melhor_fitness = float('inf')
    melhor_individuo = None
    melhor_geracao = 0

    print("População inicial:")
    for i, individuo in enumerate(populacao):
        print(f"Indivíduo {i}: {individuo}, Fitness: {fitness[i]}")
    print("\n")

    # Loop principal do algoritmo genético
    for geracao in range(geracoes):
        populacao_selecionada = selecao_torneio_min(populacao, fitness, tamanho_torneio)

        nova_populacao = []
        for i in range(0, len(populacao_selecionada), 2):
            pai = populacao_selecionada[i]
            mae = populacao_selecionada[i + 1]
            filhos = cruzamento_uniforme(pai, mae, chance_de_cruzamento)
            nova_populacao.extend(filhos)

        mutacao_simples_him(nova_populacao, chance_de_mutacao, intervalo_numerico)

        fitness_nova_populacao = calcular_fitness_him(nova_populacao)

        populacao = nova_populacao
        fitness = fitness_nova_populacao

        for i, individuo in enumerate(populacao):
            if fitness[i] < melhor_fitness:
                melhor_fitness = fitness[i]
                melhor_individuo = individuo
                melhor_geracao = geracao

    return melhor_individuo, melhor_fitness, melhor_geracao            

Com as funções definidas, o algoritmo é executado e o resultado é impresso na tela. O resultado é o melhor indivíduo encontrado, ou seja, o que possui o menor valor da função de Himmelblau.

Como se trata de uma função simples, o algoritmo é capaz de encontrar os mínimos globais em poucos segundos. O tempo de execução varia de acordo com a quantidade de gerações e a quantidade de indivíduos.

Primeiramente, testaremos com uma população de 10 indivíduos e 10 gerações, e logo após, com uma população maior. Assim, podemos observar a diferença de tempo de execução e a qualidade do resultado encontrado.

In [2]:
tamanho_populacao = 10

tamanho_torneio = 3
chance_de_cruzamento = 0.1
chance_de_mutacao = 0.1
intervalo_numerico = 0.5
geracoes = 10

melhor_individuo, melhor_fitness, melhor_geracao = executar_algoritmo_genetico(tamanho_populacao, tamanho_torneio, chance_de_cruzamento, chance_de_mutacao, intervalo_numerico, geracoes)

# Exibe o melhor indivíduo encontrado
print("Melhor indivíduo encontrado:")
print(f"Indivíduo: {melhor_individuo}, Valor da função: {melhor_fitness}")
print(f"Geração: {melhor_geracao + 1}")


População inicial:
Indivíduo 0: [2.7885359691576745, -9.49978489554666], Fitness: 7563.822830879992
Indivíduo 1: [-4.499413632617615, -5.5357852370235445], Fitness: 380.3065643808491
Indivíduo 2: [4.729424283280249, 3.533989748458225], Fitness: 326.4709294963446
Indivíduo 3: [7.843591354096908, -8.261223347411677], Fitness: 6559.588838381173
Indivíduo 4: [-1.561563606294591, -9.404055611238594], Fitness: 6702.729308614236
Indivíduo 5: [-5.627240503927933, 0.10710576206724731], Fitness: 590.6727204517896
Indivíduo 6: [-9.469280606322727, -6.02324698626703], Fitness: 5669.599796416188
Indivíduo 7: [2.997688755590463, 0.8988296120643327], Fitness: 11.447594745153678
Indivíduo 8: [-5.591187559186066, 1.7853136775181753], Fitness: 574.4888843698982
Indivíduo 9: [6.1886091335565325, -9.87002480643878], Fitness: 9636.484104918196


Melhor indivíduo encontrado:
Indivíduo: [2.451369544625778, 1.1877336010094774], Valor da função: 11.447594745153678
Geração: 1


Podemos ver que o resultado encontrado é o indivíduo $\sim[2.451, -1.187]$, em que a função vale $\sim11.447$. Podemos tentar melhorar isso! Por enquanto, vamos ver se apenas mudando a quantidade de gerações e indivíduos conseguimos um resultado melhor. Tentaremos com 20 indivíduos e 1000 gerações.

In [3]:
tamanho_populacao = 20
geracoes = 1000

melhor_individuo, melhor_fitness, melhor_geracao = executar_algoritmo_genetico(tamanho_populacao, tamanho_torneio, chance_de_cruzamento, chance_de_mutacao, intervalo_numerico, geracoes)

# Exibe o melhor indivíduo encontrado
print("Melhor indivíduo encontrado:")
print(f"Indivíduo: {melhor_individuo}, Valor da função: {melhor_fitness}")
print(f"Geração: {melhor_geracao + 1}")


População inicial:
Indivíduo 0: [2.753228735523125, 5.316273685943308], Fitness: 580.364870668243
Indivíduo 1: [0.42599625655964246, 2.5349687396356266], Fitness: 68.63922601537875
Indivíduo 2: [-4.508051061649234, -8.450332922705284], Fitness: 3588.7797559944947
Indivíduo 3: [-4.28543698273695, -4.565697858356308], Fitness: 99.23258355754045
Indivíduo 4: [-3.605808631624754, 0.8030444503691285], Fitness: 107.08755735698728
Indivíduo 5: [-7.232518769676886, -5.374770405436264], Fitness: 1506.080141151604
Indivíduo 6: [3.878996245981046, 4.128382833891045], Fitness: 260.667688334318
Indivíduo 7: [-8.715422985722439, -1.8480126066682665], Fitness: 4134.242666877269
Indivíduo 8: [0.852222810078306, -1.6845153179368815], Fitness: 153.95662993980463
Indivíduo 9: [-5.863312209711795, -1.5971296445314884], Fitness: 580.7724797697829
Indivíduo 10: [8.096769566803538, 1.6815882840845031], Fitness: 3178.2567754247493
Indivíduo 11: [3.9104597299593618, 7.134640646078687], Fitness: 2416.6973745309

Podemos perceber que agora temos o indivíduo $\sim[2.827, 1.892]$ e o valor da função $\sim0.0004$, que já está muito mais próximo do mínimo! Como o algoritmo está rodando bem rápido, podemos aumentar a quantidade de gerações e indivíduos para ver se conseguimos um resultado ainda melhor. Vamos tentar com 100 indivíduos e 10000 gerações.

In [4]:
tamanho_populacao = 100
geracoes = 10000

melhor_individuo, melhor_fitness, melhor_geracao = executar_algoritmo_genetico(tamanho_populacao, tamanho_torneio, chance_de_cruzamento, chance_de_mutacao, intervalo_numerico, geracoes)

# Exibe o melhor indivíduo encontrado
print("Melhor indivíduo encontrado:")
print(f"Indivíduo: {melhor_individuo}, Valor da função: {melhor_fitness}")
print(f"Geração: {melhor_geracao + 1}")


População inicial:
Indivíduo 0: [-3.4628816248397998, 8.776490752958864], Fitness: 4526.168493259324
Indivíduo 1: [-4.42641762908083, -2.021634385665953], Fitness: 97.05208920461783
Indivíduo 2: [6.865219056337811, 2.3303263886902688], Fitness: 1507.3353313098291
Indivíduo 3: [-1.46774357366899, -4.573410759741828], Fitness: 335.03453517484706
Indivíduo 4: [-2.9219394029039103, -7.827478893427817], Fitness: 2742.443296956511
Indivíduo 5: [-7.730927069644061, -8.527583423197893], Fitness: 4981.924781759967
Indivíduo 6: [-8.277584277839402, -7.966118925468891], Fitness: 4776.882415975315
Indivíduo 7: [-4.180613500108579, 0.29269826004681754], Fitness: 168.93370123343027
Indivíduo 8: [2.401845669727244, 3.8928648938928205], Fitness: 113.22523427946497
Indivíduo 9: [8.368172756327795, 1.4335355653984916], Fitness: 3667.1118417372013
Indivíduo 10: [-3.326913429752933, -7.76572059624127], Fitness: 2557.2001834302046
Indivíduo 11: [-2.3711561888682446, -5.062666843522603], Fitness: 373.368923

Agora sim! Obtivemos um resultado em que a função vale $\sim6.280 * 10^-07$, ou seja, muito muito próximo do mínimo global. O indivíduo encontrado foi $\sim[3.34152, 2.00007]$. Agora vamos comparar com fontes externas para ver se o resultado é satisfatório. Existem quatro mínimos globais para a função de Himmelblau, que são $^{[1]}$ $^{[2]}$:
- $f(-3.779, -3.283) = 0$ 
- $f(-2.805, 3.131) = 0$
- $f(3.0, 2.0) = 0$
- $f(3.584, -1.848) = 0$

Podemos perceber que apesar de não termos encontrado os mínimos globais, o resultado encontrado está muito próximo do mínimo global. O algoritmo genético é uma ótima ferramenta para resolver problemas de otimização, e nesse caso, conseguimos um resultado satisfatório em poucos segundos. É provável que se aumentássemos ainda mais a quantidade de gerações e indivíduos, conseguiríamos um resultado ainda melhor. Porém, o tempo de execução aumentaria consideravelmente.

## Conclusão

O algoritmo genético é uma boa técnica para resolver problemas de otimização, e neste caso, foi capaz de encontrar os mínimos globais da função de Himmelblau em poucos segundos. 

Podemos perceber que a quantidade de gerações e indivíduos influencia diretamente no resultado encontrado, e que aumentar esses valores pode levar a resultados ainda melhores.

## Referências

$^{[1]}$ WIKIPEDIA CONTRIBUTORS. Himmelblau’s function. Disponível em: <https://en.wikipedia.org/wiki/Himmelblau%27s_function>. 

$^{[2]}$ INDUSMIC PRIVATE LIMITED. Python Implementation of HIMMELBLAU FUNCTION. Disponível em: <https://www.indusmic.com/post/himmelblau-function>. 